# [CONFIG] Configuração básica

## Básicos (numpy, pandas, math, display, locale, time, random, json)

In [1]:
!python -m pip install pandas
!python -m pip install openpyxl
!python -m pip install xlrd
!python -m pip install jupyter
!python -m pip install numpy
# !python -m pip install random
!python -m pip install IPython
!python -m pip install json
# !sudo dpkg-reconfigure locales


from IPython.display import display
import json
import pandas as pd
import math
import numpy as np
import locale
# locale.setlocale(locale.LC_ALL, "pt_BR.UTF-8")  # Use "" for auto, or force e.g. to "en_US.UTF-8"

import time
from datetime import datetime, timedelta, date
import random

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.7 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


## Visualização (matplotlib, seaborn)

In [2]:
!python -m pip install matplotlib
!python -m pip install seaborn

import matplotlib.pyplot as plt
# import seaborn as sns

## Automação (selenium, schedule, BeautifulSoup)

In [3]:
# !python -m pip install selenium

# from selenium import webdriver
# # from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC

In [4]:
# !python -m pip install schedule
# import schedule

## Web (BeautifulSoup, requests)

In [5]:
!python -m pip install BeautifulSoup
from bs4 import BeautifulSoup

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
!python -m pip install requests
import requests

## Drive (para ler e escrever arquivos)

In [7]:
from google.colab import drive
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
drive.mount('/content/drive')

Mounted at /content/drive


# Funções

## Elimina colunas NA

In [8]:
def elimina_colunas_NA(
    base,
    imprime_colunas_vazias = False,
    imprime_colunas_completas = False,
    imprime_colunas_parciais = False,
    remove_da_base = True,
    retorna_parciais = False
    ):

    tamanho_da_base = len(base)

    colunas_vazias = []
    colunas_completas = []
    colunas_parciais = []

    for coluna in base.columns:
        tamanho_da_coluna = len(base[base[coluna].isna()])
        if tamanho_da_coluna == tamanho_da_base:
            # print(coluna)
            colunas_vazias.append(coluna)
        elif tamanho_da_coluna == 0:
            colunas_completas.append(coluna)
        else:
            colunas_parciais.append(coluna)

    if imprime_colunas_vazias == True:
        if len(colunas_vazias) == 0:
            print("Não há colunas NA")
        else:
            print("Colunas vazias: " + str(colunas_vazias))

    if imprime_colunas_completas == True:
        if len(colunas_completas) == 0:
            print("Não há colunas completas")
        else:
            print("Colunas completas: " + str(colunas_completas))

    if imprime_colunas_parciais == True:
        if len(colunas_parciais) == 0:
            print("Não há colunas parciais")
        else:
            print("Colunas parciais: " + str(colunas_parciais))

    if remove_da_base == True:
        base = base.drop(colunas_vazias, axis = 1)

    if retorna_parciais == True:
        return [base, colunas_parciais]
    else:
        return base

## Agrupar cada coluna

In [9]:
def agrupamento_cada_coluna(
    bd,
    coluna_completa,
    colunas_ignoradas = [],
    ascending = False,
    imprime_tabelas = True,
    ):

    from IPython.display import display

    campos_com_erro = []
    dict_campos = {}

    colunas = bd.columns.drop(coluna_completa)

    if len(colunas_ignoradas) > 0:
        colunas = colunas.drop(colunas_ignoradas)

    for coluna in colunas:
        try:
            temp_coluna = bd.fillna("(vazio)").groupby(coluna).count()[[coluna_completa]].rename(columns = {coluna_completa: "Quantidade"}).sort_values("Quantidade", ascending = ascending)

            if ascending == False:
                temp_coluna["%"] = temp_coluna["Quantidade"]/temp_coluna["Quantidade"].sum()
                temp_coluna["% acumulado"] = temp_coluna["%"].cumsum()

            if imprime_tabelas == True:
                display(temp_coluna)

            dict_campos[coluna] = temp_coluna
            limpa(temp_coluna)

        except:
            campos_com_erro.append(coluna)
            #print("Campo " + coluna + " deu erro =/")

    return [campos_com_erro, dict_campos]

## Análise exploratória básica

In [10]:
def analise_exploratoria(
    bd,
    imprime_todas_colunas = False,
    imprime_info_colunas = True,
    imprime_colunas_vazias = False,
    imprime_colunas_completas = False,
    imprime_colunas_parciais = False,
    remove_da_base = True,
    retorna_parciais = False,
    detalhar_colunas_parciais = True,
    colunas_ignoradas = []
    ):

    # COMEÇANDO PELAS COLUNAS DISPONÍVEIS E INFO
    if imprime_todas_colunas == True:
        display(bd.columns)

    if imprime_info_colunas == True:
        for i in range(int(np.ceil(len(bd.columns)/20))):
            display(bd.iloc[:, (i*20):min((i+1)*20, len(bd.columns))].info())

    # DETALHAMENTO DE QUAIS COLUNAS SÃO NA OU PARCIAIS
    if retorna_parciais == True:
        [bd_semNA, colunas_parciais] = elimina_colunas_NA(
            bd,
            imprime_colunas_vazias = imprime_colunas_vazias,
            imprime_colunas_completas = imprime_colunas_completas,
            imprime_colunas_parciais = imprime_colunas_parciais,
            remove_da_base = remove_da_base,
            retorna_parciais = retorna_parciais
        )

        if detalhar_colunas_parciais == True:
            for coluna in colunas_parciais:
                print("# " + coluna + ": " + str(len(colunas_parciais[colunas_parciais[coluna].isna()])))
    else:
        colunas_parciais = []

        bd_semNA = elimina_colunas_NA(
            bd,
            imprime_colunas_vazias = imprime_colunas_vazias,
            imprime_colunas_completas = imprime_colunas_completas,
            imprime_colunas_parciais = imprime_colunas_parciais,
            remove_da_base = remove_da_base,
            retorna_parciais = retorna_parciais
        )

    [campos_com_erro, colunas_agrupadas] = agrupamento_cada_coluna(
        bd.reset_index(),
        coluna_completa = bd_semNA.drop(colunas_parciais, axis = 1).reset_index().columns[0],
        colunas_ignoradas = colunas_ignoradas
    )

    if retorna_parciais == True:
        return [bd_semNA, colunas_parciais, colunas_agrupadas, campos_com_erro]
    else:
        return [bd_semNA, colunas_agrupadas, campos_com_erro]

## Completa dígitos (padronizar chaves, CPF, etc)

In [11]:
def completa_digitos(texto, qtd):
    quantidade_zeros = qtd - len(texto)

    contador = quantidade_zeros
    while contador > 0:
        texto = "0" + texto
        contador = contador - 1

    # print(len(texto))

    return texto

## Exemplo de uso
# df["coluna_nova"] = df.apply(lambda x: completa_digitos(x["coluna_referencia"], 2), axis = 1)

## Histograma padrão

In [12]:
def histograma_padrao(
  dados_x,
  titulo = np.nan,
  dados_indice = np.nan,
  tipo = "hist",
  grid = False,
  legend = True,
  linha_media = np.nan,
  qtd_bins = 10,
  espessura = 0.8,
  limite_x_min = np.nan, limite_x_max = np.nan,
  limite_y_min = np.nan, limite_y_max = np.nan,
  ):

  plt.figure(figsize = (20,8))

  if not(pd.isnull(titulo)):
      plt.title(titulo, fontsize = 16)

  if tipo == "hist":
      n, bins, edges = plt.hist(
          dados_x,
          bins = qtd_bins,
          rwidth = espessura
          )
      plt.xticks(bins)

  elif tipo == "bar":
      plt.bar(
          dados_indice,
          dados_x,
          width = espessura
          )



  if not(math.isnan(linha_media)):
      plt.axvline(x=linha_media, color = "black", label='Média')

  if legend == True:
      plt.legend()

  plt.grid(grid)

  if (not(math.isnan(limite_x_max))) & (not(math.isnan(limite_x_min))):
      # print(limite_x_min)
      # print(~math.isnan(limite_x_max))
      plt.xlim(limite_x_min, limite_x_max)

  if (not(math.isnan(limite_y_max))) & (not(math.isnan(limite_y_min))):
      plt.ylim(limite_y_min, limite_y_max)

  plt.show()

## Gráfico com rótulos

In [13]:
def grafico_com_rotulos(
    dados, coluna_x, coluna_y,
    tipo_grafico = "bar", titulo_grafico = "", tamanho_grafico = (20, 8),
    fig = None, ax = None,
    tamanho_fonte = 12, grossura_barra = 1,
    cor_x = "b", rotacao_x = 0,
    formato_eixo_y = "{x:.2f}",
    formato_y = "{:.2f}", rotacao_y = 0, limite_min_y = None, limite_max_y = None,
    xytext_label_y = (0,10),
  ):

  # dados_plot = dados_cliente[dados_cliente["% Faturamento Acumulado"] < 0.40]

  if fig is None:
    fig = plt.figure(figsize = tamanho_grafico)

  plt.clf()
  plt.rcParams.update({'font.size': tamanho_fonte})


  ys = dados[coluna_y]
  xs = np.arange(len(dados[coluna_x]))

  if (tipo_grafico == "bar") | (tipo_grafico == "line"):
    if (tipo_grafico == "bar"):
      plt.bar(
          x = xs,
          height = ys,
          width = grossura_barra,
          label = coluna_y,
      )
    elif (tipo_grafico == "line"):
      plt.plot(
        xs,
        ys,
        cor_x + "o-",
        label = coluna_y,
      )

    # display(cor_x + "o-")

    ax1 = plt.gca()
    if (limite_min_y is not None) & (limite_max_y is not None):
      ax1.set_ylim([min(dados[coluna_y])*limite_min_y, max(dados[coluna_y])*limite_max_y])

    ax1.set_ylabel(coluna_y, color = cor_x)
    ax1.set_xlabel(coluna_x)

    for x,y in zip(xs,ys):
        label = formato_y.format(y)

        plt.annotate(label, # this is the text
                    (x,y), # these are the coordinates to position the label
                    textcoords="offset points", # how to position the text
                    xytext = xytext_label_y, # distance from text to points (x,y)
                    ha='center', # horizontal alignment can be left, right or center
                    color = cor_x,
                    rotation = rotacao_y)


    ax1.xaxis.set_tick_params(rotation = rotacao_x)
    ax1.yaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter(formato_eixo_y))

    # fig.legend()
    plt.xticks(xs, dados[coluna_x])

  elif tipo_grafico == "barh":
    plt.barh(
        y = xs,
        width = ys,
        height = grossura_barra,
        # label = coluna_y,
    )
    ax1 = plt.gca()
    ax1.set_yticks(xs, dados[coluna_x])

    ax1.set_ylabel(coluna_x)
    ax1.set_xlabel(coluna_y, color = cor_x)
    ax1.invert_yaxis()

    if (limite_min_y is not None) & (limite_max_y is not None):
      ax1.set_xlim([min(dados[coluna_y])*limite_min_y, max(dados[coluna_y])*limite_max_y])

    for x,y in zip(xs,ys):
      label = formato_y.format(y)

      plt.annotate(label, # this is the text
                (y, x), # these are the coordinates to position the label
                textcoords="offset points", # how to position the text
                xytext = (xytext_label_y[1], xytext_label_y[0]), # distance from text to points (x,y)
                ha='center', # horizontal alignment can be left, right or center
                color = cor_x,
                rotation = rotacao_y)

    ax1.xaxis.set_major_formatter(matplotlib.ticker.StrMethodFormatter(formato_eixo_y))

  plt.title(titulo_grafico)
  plt.show()

## GridSearch

In [14]:
def treina_modelo_grid(
  x_train,
  y_train,
  modelo,
  param_grid,
  cv = 10,
  scoring = "neg_mean_absolute_error",
  tipo = "grid",
  random_state_seed = 1082141,
  n_iter = 50
):
  from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

  if tipo == "grid":
      grid = GridSearchCV(
          modelo,
          param_grid,
          cv = cv,
          scoring = scoring,
          return_train_score = True,
      )

  elif tipo == "randomized":
      # print("Busca randomizada em " + str(n_iter) + " combinações de parâmetros.")
      grid = RandomizedSearchCV(
          modelo,
          param_grid,
          cv = cv,
          scoring = scoring,
          return_train_score = True,
          random_state = random_state_seed,
          n_iter = n_iter,
      )

  grid.fit(x_train, y_train)

  return grid

### Definimos range de hiperparâmetros
# n_neighbors_range = range(10, 20)
# metric_range = ["cityblock", "cosine", "euclidean", "haversine", "l1", "l2", "manhattan", "nan_euclidean", "minkowski"]
# algorithm_range = ["auto", "ball_tree", "kd_tree", "brute"]

# param_grid = dict(algorithm = algorithm_range)
# # display(param_grid)

# grid_melhor_algorithm = treina_modelo_grid(
#     dados_treino_X,
#     dados_treino_y,
#     KNeighborsClassifier(n_neighbors = 12, metric = "euclidean"),
#     param_grid,
#     cv = 5,
#     scoring = "accuracy",
#     tipo = "grid",
#     random_state_seed = numero_aleatorio,
#     # n_iter = 50
# )

# # print(grid.best_score_)
# # grid.best_estimator_
# tabela_resultados = pd.concat([
#   pd.json_normalize(pd.DataFrame(grid_melhor_algorithm.cv_results_)["params"]),
#   pd.DataFrame(grid_melhor_algorithm.cv_results_)[["rank_test_score", "mean_test_score"]]
# ], axis = 1).set_index("rank_test_score").sort_index()


# plt.figure(figsize = (15,8))
# display(tabela_resultados)
# plt.bar(x = tabela_resultados["algorithm"], height = tabela_resultados["mean_test_score"]);

# Ler da pasta (caso já tenha sido lido dos sites antes)

## Variável global (reler dados)

In [15]:
# var_reler_dados = False
var_reler_dados = True

## Leitura da pasta

In [16]:
if var_reler_dados == False:
  base_jogos = pd.read_excel(
      '/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Base Jogos.xlsx',
      # encoding = 'ISO-8859-1',
      # low_memory=False,
      index_col = 0
    )
  display(base_jogos)

# Leitura (site Academia Apostas Brasil)

## Brasileirão

In [51]:
brasileirao_resultado = []
# print(i+1)
rodada = "13"
print("Leitura da Rodada: " + rodada)

brasileirao_url = 'https://www.academiadasapostasbrasil.com/stats/competition/brasil/26/NR2zmkLnBQ08o/og0WZv4p7QLJv/0/' + rodada
brasileirao_response_rodada = requests.get(brasileirao_url)

# Create a BeautifulSoup object with the HTML content
brasileirao_soup_rodada = BeautifulSoup(brasileirao_response_rodada.content, 'html.parser')
# brasileirao_soup_rodada

# Se quiser exportar e ler os dados offline
# with open('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Site Brasileirão.html', "w") as file:
#   file.write(str(brasileirao_soup))

brasileirao_jogos_rodada = brasileirao_soup_rodada. \
  find_all('table', class_='competition-rounds competition-half-padding')[0]. \
  find_all("tr", class_=lambda s: "even" in s or "odd" in s)

for jogo in brasileirao_jogos_rodada:
  # O site tem uma tag a mais "Previsão" para jogos que não rolaram ainda
  posicao = 0
  if jogo.find_all("a")[0].text.strip() == "Previsão":
    posicao = 1

  try:
    data_jogo = jogo.find("td", class_="darker tipsy-active nowrap").get("title")
  except:
    data_jogo = jogo.find("td", class_="darker tipsy-active nowrap gameinlive").get("title")
  link_jogo = jogo.find_all("a")[3 + posicao].get("href")
  timeA_nome = jogo.find_all("a")[0 + posicao].text.strip()
  timeB_nome = jogo.find_all("a")[2 + posicao].text.strip()
  resultado = jogo.find_all("a")[1 + posicao].text.strip()
  if int(rodada) <= 19:
    ida_ou_volta = "Ida"
  else:
    ida_ou_volta = "Volta"

  # Add the dictionary to the data list
  brasileirao_resultado.append({
      'data_jogo': data_jogo,
      'link_jogo': link_jogo,
      # 'local_jogo': local_jogo,
      # 'timeA_sigla': timeA_sigla,
      'timeA_nome': timeA_nome,
      # 'timeB_sigla': timeB_sigla,
      'timeB_nome': timeB_nome,
      'resultado': resultado,
      'fase': "Rodada " + rodada,
      'ida ou volta': ida_ou_volta,
      'torneio': "Brasileirão",
  })

# brasileirao_resultado

# Convert the data to a pandas DataFrame
brasileirao_base_resultado = pd.DataFrame(brasileirao_resultado)

display(brasileirao_base_resultado.head())
display(brasileirao_base_resultado.tail())

Leitura da Rodada: 13


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,01/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Fluminense,1-0,Rodada 13,Ida,Brasileirão
1,01/07/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Bahia,Grêmio,1-2,Rodada 13,Ida,Brasileirão
2,01/07/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Fortaleza,2-0,Rodada 13,Ida,Brasileirão
3,01/07/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Internacional,Cruzeiro,0-0,Rodada 13,Ida,Brasileirão
4,02/07/2023 11:00h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,RB Bragantino,0-1,Rodada 13,Ida,Brasileirão


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
5,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Botafogo,Vasco,2-0,Rodada 13,Ida,Brasileirão
6,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Palmeiras,2-2,Rodada 13,Ida,Brasileirão
7,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Atlético-MG,América-MG,2-2,Rodada 13,Ida,Brasileirão
8,02/07/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Cuiabá,Santos,3-0,Rodada 13,Ida,Brasileirão
9,03/07/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Goiás,Coritiba,vs,Rodada 13,Ida,Brasileirão


In [52]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  brasileirao_resultado = []

  for i in range(38):
    # print(i+1)
    rodada = str(i+1)
    print("Leitura da Rodada: " + rodada)

    brasileirao_url = 'https://www.academiadasapostasbrasil.com/stats/competition/brasil/26/NR2zmkLnBQ08o/og0WZv4p7QLJv/0/' + rodada
    brasileirao_response_rodada = requests.get(brasileirao_url)

    # Create a BeautifulSoup object with the HTML content
    brasileirao_soup_rodada = BeautifulSoup(brasileirao_response_rodada.content, 'html.parser')
    # brasileirao_soup_rodada

    # Se quiser exportar e ler os dados offline
    # with open('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Site Brasileirão.html', "w") as file:
    #   file.write(str(brasileirao_soup))

    brasileirao_jogos_rodada = brasileirao_soup_rodada. \
      find_all('table', class_='competition-rounds competition-half-padding')[0]. \
      find_all("tr", class_=lambda s: "even" in s or "odd" in s)

    for jogo in brasileirao_jogos_rodada:
      # O site tem uma tag a mais "Previsão" para jogos que não rolaram ainda
      posicao = 0
      if jogo.find_all("a")[0].text.strip() == "Previsão":
        posicao = 1

      try:
        data_jogo = jogo.find("td", class_="darker tipsy-active nowrap").get("title")
      except:
        data_jogo = jogo.find("td", class_="darker tipsy-active nowrap gameinlive").get("title")
      link_jogo = jogo.find_all("a")[3 + posicao].get("href")
      timeA_nome = jogo.find_all("a")[0 + posicao].text.strip()
      timeB_nome = jogo.find_all("a")[2 + posicao].text.strip()
      resultado = jogo.find_all("a")[1 + posicao].text.strip()
      if int(rodada) <= 19:
        ida_ou_volta = "Ida"
      else:
        ida_ou_volta = "Volta"

      # Add the dictionary to the data list
      brasileirao_resultado.append({
          'data_jogo': data_jogo,
          'link_jogo': link_jogo,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome,
          'resultado': resultado,
          'fase': "Rodada " + rodada,
          'ida ou volta': ida_ou_volta,
          'torneio': "Brasileirão",
      })

  # brasileirao_resultado

  # Convert the data to a pandas DataFrame
  brasileirao_base_resultado = pd.DataFrame(brasileirao_resultado)

  display(brasileirao_base_resultado.head())
  display(brasileirao_base_resultado.tail())

Leitura da Rodada: 1
Leitura da Rodada: 2
Leitura da Rodada: 3
Leitura da Rodada: 4
Leitura da Rodada: 5
Leitura da Rodada: 6
Leitura da Rodada: 7
Leitura da Rodada: 8
Leitura da Rodada: 9
Leitura da Rodada: 10
Leitura da Rodada: 11
Leitura da Rodada: 12
Leitura da Rodada: 13
Leitura da Rodada: 14
Leitura da Rodada: 15
Leitura da Rodada: 16
Leitura da Rodada: 17
Leitura da Rodada: 18
Leitura da Rodada: 19
Leitura da Rodada: 20
Leitura da Rodada: 21
Leitura da Rodada: 22
Leitura da Rodada: 23
Leitura da Rodada: 24
Leitura da Rodada: 25
Leitura da Rodada: 26
Leitura da Rodada: 27
Leitura da Rodada: 28
Leitura da Rodada: 29
Leitura da Rodada: 30
Leitura da Rodada: 31
Leitura da Rodada: 32
Leitura da Rodada: 33
Leitura da Rodada: 34
Leitura da Rodada: 35
Leitura da Rodada: 36
Leitura da Rodada: 37
Leitura da Rodada: 38


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,15/04/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Palmeiras,Cuiabá,2-1,Rodada 1,Ida,Brasileirão
1,15/04/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,América-MG,Fluminense,0-3,Rodada 1,Ida,Brasileirão
2,15/04/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Botafogo,São Paulo,2-1,Rodada 1,Ida,Brasileirão
3,15/04/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Fortaleza,Internacional,1-1,Rodada 1,Ida,Brasileirão
4,15/04/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Goiás,2-0,Rodada 1,Ida,Brasileirão


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
375,02/12/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Santos,Fortaleza,vs,Rodada 38,Volta,Brasileirão
376,02/12/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Internacional,Botafogo,vs,Rodada 38,Volta,Brasileirão
377,02/12/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Goiás,América-MG,vs,Rodada 38,Volta,Brasileirão
378,02/12/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Fluminense,Grêmio,vs,Rodada 38,Volta,Brasileirão
379,02/12/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Coritiba,Corinthians,vs,Rodada 38,Volta,Brasileirão


In [53]:
# # Por curiosidade, um jogo lá do fim do campeonato já tem a página pronta com todas as análises
# temp_site = requests.get(brasileirao_base_resultado.iloc[379]["link_jogo"])
# temp_site_soup = BeautifulSoup(temp_site.content, 'html.parser')
# with open('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/temp.html', "w") as file:
#   file.write(str(temp_site_soup))

## Copa do Brasil

In [54]:
if var_reler_dados == True:
  copabrasil_url = {
    "1ª Fase": "https://www.academiadasapostasbrasil.com/stats/competition/brasil/231/8pj1Q9wzoZb3k/9jJzZx4k6ZR0v",
    "2ª Fase": "https://www.academiadasapostasbrasil.com/stats/competition/brasil/231/8pj1Q9wzoZb3k/pqdPZDeldm81J",
    "3ª Fase": "https://www.academiadasapostasbrasil.com/stats/competition/brasil/231/8pj1Q9wzoZb3k/XaBgZEr4jmE2J",
    "Oitavas de Final": "https://www.academiadasapostasbrasil.com/stats/competition/brasil/231/8pj1Q9wzoZb3k/l2RBY0rPWmw0J",
    "Quartas de Final": "https://www.academiadasapostasbrasil.com/stats/competition/brasil/231/8pj1Q9wzoZb3k/3l2OZXjxRQ7Wb",
  }

  # Initialize a list to store the extracted data
  copabrasil_resultado = []

  # for i in copabrasil_url:
  #   print(i + ": " + copabrasil_url[i])
  #   copabrasil_response_fase = requests.get(copabrasil_url[i])
  #   copabrasil_soup_fase = BeautifulSoup(copabrasil_response_fase.content, 'html.parser')
  #   nome_arquivo_copa_brasil_exportacao = '/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Site Copa do Brasil (' + i + ').html'
  #   with open(nome_arquivo_copa_brasil_exportacao, "w") as file:
  #     file.write(str(copabrasil_soup_fase))

  for fase in copabrasil_url:
    print("Leitura da Fase: " + fase)
    copabrasil_response_fase = requests.get(copabrasil_url[fase])

    copabrasil_soup_fase = BeautifulSoup(copabrasil_response_fase.content, 'html.parser')
    copabrasil_jogos_fase = copabrasil_soup_fase. \
      find_all('table', class_='competition-rounds competition-half-padding')[0]. \
      find_all("tr", class_= ["even", "odd"])

    # display(copabrasil_jogos_fase[0:2])
    # display(len(copabrasil_jogos_fase))


    ida_ou_volta = "Ida"
    for jogo in copabrasil_jogos_fase:
      posicao = 0

      try:
        # display(jogo.find_all("td", class_= "darker tipsy-active nowrap")[0].get("title"))
        data_jogo = jogo.find_all("td", class_= "darker tipsy-active nowrap")[0].get("title")
      except:
        try:
          # display(jogo.find_all("td", class_= "darker tipsy-active nowrap gameinlive")[0].get("title"))
          data_jogo = jogo.find_all("td", class_= "darker tipsy-active nowrap gameinlive")[0].get("title")
        except:
          continue

      # # display(jogo)
      if jogo.find_all("a")[0].text.strip() == "Previsão":
        posicao = 1

      # display(jogo.find_all("a")[3 + posicao].get("href"))
      link_jogo = jogo.find_all("a")[3 + posicao].get("href")
      # display(jogo.find_all("a")[0 + posicao].text.strip())
      timeA_nome = jogo.find_all("a")[0 + posicao].text.strip()
      # display(jogo.find_all("a")[2 + posicao].text.strip())
      timeB_nome = jogo.find_all("a")[2 + posicao].text.strip()
      # display(jogo.find_all("a")[1 + posicao].text.strip())
      resultado = jogo.find_all("a")[1 + posicao].text.strip()
      # print("")

      copabrasil_resultado.append({
            'data_jogo': data_jogo,
            'link_jogo': link_jogo,
            # 'local_jogo': local_jogo,
            # 'timeA_sigla': timeA_sigla,
            'timeA_nome': timeA_nome,
            # 'timeB_sigla': timeB_sigla,
            'timeB_nome': timeB_nome,
            'resultado': resultado,
            'fase': fase,
            'ida ou volta': ida_ou_volta,
            'torneio': "Copa do Brasil",
        })

      # display(ida_ou_volta)
      if (fase != "1ª Fase") * (fase != "2ª Fase"):
        if ida_ou_volta == "Ida":
          ida_ou_volta = "Volta"
        else:
          ida_ou_volta = "Ida"

      # copabrasil_resultado

  # Convert the data to a pandas DataFrame
  copabrasil_base_resultado = pd.DataFrame(copabrasil_resultado)

  display(copabrasil_base_resultado.head())
  display(copabrasil_base_resultado.tail())

Leitura da Fase: 1ª Fase
Leitura da Fase: 2ª Fase
Leitura da Fase: 3ª Fase
Leitura da Fase: Oitavas de Final
Leitura da Fase: Quartas de Final


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,21/02/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Marcílio Dias,Chapecoense,1-0,1ª Fase,Ida,Copa do Brasil
1,22/02/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Falcon,Volta Redonda,1-3,1ª Fase,Ida,Copa do Brasil
2,22/02/2023 17:00h,https://www.academiadasapostasbrasil.com/stats...,Nova Mutum,Londrina,4-2,1ª Fase,Ida,Copa do Brasil
3,22/02/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Caucaia,Tombense,0-0,1ª Fase,Ida,Copa do Brasil
4,22/02/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,São Raimundo R,Cuiabá,4-3,1ª Fase,Ida,Copa do Brasil


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
110,13/07/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Palmeiras,São Paulo,vs,Quartas de Final,Volta,Copa do Brasil
111,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,América-MG,Corinthians,vs,Quartas de Final,Ida,Copa do Brasil
112,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,América-MG,vs,Quartas de Final,Volta,Copa do Brasil
113,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Athletico PR,vs,Quartas de Final,Ida,Copa do Brasil
114,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Flamengo,vs,Quartas de Final,Volta,Copa do Brasil


## Libertadores

In [55]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  libertadores_fase_grupos_resultado = []

  for i in range(6):
    # print(i+1)
    rodada = str(i+1)

    libertadores_url_fase_grupos_rodada = 'https://www.academiadasapostasbrasil.com/stats/competition/america-do-sul/241/G1L9QaknaZoPd/2xVym4AL2QRqB/0/'+rodada
    libertadores_response_fase_grupos_rodada = requests.get(libertadores_url_fase_grupos_rodada)

    # Create a BeautifulSoup object with the HTML content
    libertadores_soup_fase_grupos_rodada = BeautifulSoup(libertadores_response_fase_grupos_rodada.content, 'html.parser')

    # libertadores_soup_fase_grupos

    # libertadores_fase_grupos_rodada = libertadores_soup_fase_grupos_rodada.find_all('tr', class_='even')
    # libertadores_fase_grupos_rodada = libertadores_soup_fase_grupos_rodada.find_all('td', class_='nowrap')
    libertadores_fase_grupos_rodada = libertadores_soup_fase_grupos_rodada.\
      find_all('table', class_='competition-rounds competition-half-padding')

    # libertadores_fase_grupos_rodada = libertadores_soup_fase_grupos_rodada.\
    #   find_all('table', class_='competition-rounds competition-half-padding')[0].\
    #   find_all('a')

    # len(libertadores_fase_grupos_rodada)
    # display(libertadores_fase_grupos_rodada[0].find_all("a")[1].text.strip())
    # display(libertadores_fase_grupos_rodada[0].find_all("a")[6].text.strip())

    # Iterate over the div elements
    for grupo, jogo in enumerate(libertadores_fase_grupos_rodada):
      # print(chr(grupo + 65))

      data_jogo1 = jogo.find_all("td", class_="darker tipsy-active nowrap")[0].get("title")
      link_jogo1 = jogo.find_all("a")[3].get("href")
      timeA_nome1 = jogo.find_all("a")[0].text.strip()
      timeB_nome1 = jogo.find_all("a")[2].text.strip()
      resultado1 = jogo.find_all("a")[1].text.strip()
      if int(rodada) <= 3:
        ida_ou_volta1 = "Ida"
      else:
        ida_ou_volta1 = "Volta"

      # Add the dictionary to the data list
      libertadores_fase_grupos_resultado.append({
          'data_jogo': data_jogo1,
          'link_jogo': link_jogo1,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome1,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome1,
          'resultado': resultado1,
          'fase': "Grupos - Rodada " + rodada + " - Grupo " + chr(grupo + 65),
          'ida ou volta': ida_ou_volta1,
          'torneio': "Libertadores",
      })


      data_jogo2 = jogo.find_all("td", class_="darker tipsy-active nowrap")[1].get("title")
      link_jogo2 = jogo.find_all("a")[8].get("href")
      timeA_nome2 = jogo.find_all("a")[5].text.strip()
      timeB_nome2 = jogo.find_all("a")[7].text.strip()
      resultado2 = jogo.find_all("a")[6].text.strip()
      if int(rodada) <= 3:
        ida_ou_volta2 = "Ida"
      else:
        ida_ou_volta2 = "Volta"

      libertadores_fase_grupos_resultado.append({
          'data_jogo': data_jogo2,
          'link_jogo': link_jogo2,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome2,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome2,
          'resultado': resultado2,
          'fase': "Grupos - Rodada " + rodada + " - Grupo " + chr(grupo + 65),
          'ida ou volta': ida_ou_volta2,
          'torneio': "Libertadores",
      })

  # libertadores_fase_grupos_resultado

  libertadores_base_fase_grupos = pd.DataFrame(libertadores_fase_grupos_resultado)

  display(libertadores_base_fase_grupos.head())
  display(libertadores_base_fase_grupos.tail())

,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
91,29/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Colo Colo,Pereira,0-0,Grupos - Rodada 6 - Grupo F,Volta,Libertadores
92,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Alianza Lima,3-0,Grupos - Rodada 6 - Grupo G,Volta,Libertadores
93,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Libertad,Atlético-MG,1-1,Grupos - Rodada 6 - Grupo G,Volta,Libertadores
94,27/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Atl. Nacional,Patronato,0-1,Grupos - Rodada 6 - Grupo H,Volta,Libertadores
95,27/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Olimpia,Melgar,4-1,Grupos - Rodada 6 - Grupo H,Volta,Libertadores


## Sul-Americana

In [56]:
if var_reler_dados == True:
  # Initialize a list to store the extracted data
  sulamericana_fase_grupos_resultado = []

  for i in range(6):
    # print(i+1)
    rodada = str(i+1)

    sulamericana_url_fase_grupos_rodada = 'https://www.academiadasapostasbrasil.com/stats/competition/america-do-sul/297/kqxeZPb10QroL/gEyAmAr7zQ2po/0/'+rodada
    sulamericana_response_fase_grupos_rodada = requests.get(sulamericana_url_fase_grupos_rodada)

    # Create a BeautifulSoup object with the HTML content
    sulamericana_soup_fase_grupos_rodada = BeautifulSoup(sulamericana_response_fase_grupos_rodada.content, 'html.parser')
    # sulamericana_soup_fase_grupos

    sulamericana_fase_grupos_rodada = sulamericana_soup_fase_grupos_rodada.\
      find_all('table', class_='competition-rounds competition-half-padding')


    # len(sulamericana_fase_grupos_rodada)
    # display(sulamericana_fase_grupos_rodada[0].find_all("a")[1].text.strip())
    # display(sulamericana_fase_grupos_rodada[0].find_all("a")[6].text.strip())

    # Iterate over the div elements
    for grupo, jogo in enumerate(sulamericana_fase_grupos_rodada):
      # print(chr(grupo + 65))

      data_jogo1 = jogo.find_all("td", class_="darker tipsy-active nowrap")[0].get("title")
      link_jogo1 = jogo.find_all("a")[3].get("href")
      timeA_nome1 = jogo.find_all("a")[0].text.strip()
      timeB_nome1 = jogo.find_all("a")[2].text.strip()
      resultado1 = jogo.find_all("a")[1].text.strip()
      if int(rodada) <= 3:
        ida_ou_volta1 = "Ida"
      else:
        ida_ou_volta1 = "Volta"

      # Add the dictionary to the data list
      sulamericana_fase_grupos_resultado.append({
          'data_jogo': data_jogo1,
          'link_jogo': link_jogo1,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome1,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome1,
          'resultado': resultado1,
          'fase': "Grupos - Rodada " + rodada + " - Grupo " + chr(grupo + 65),
          'ida ou volta': ida_ou_volta1,
          'torneio': "Sul-Americana",
      })


      data_jogo2 = jogo.find_all("td", class_="darker tipsy-active nowrap")[1].get("title")
      link_jogo2 = jogo.find_all("a")[8].get("href")
      timeA_nome2 = jogo.find_all("a")[5].text.strip()
      timeB_nome2 = jogo.find_all("a")[7].text.strip()
      resultado2 = jogo.find_all("a")[6].text.strip()
      if int(rodada) <= 3:
        ida_ou_volta2 = "Ida"
      else:
        ida_ou_volta2 = "Volta"

      sulamericana_fase_grupos_resultado.append({
          'data_jogo': data_jogo2,
          'link_jogo': link_jogo2,
          # 'local_jogo': local_jogo,
          # 'timeA_sigla': timeA_sigla,
          'timeA_nome': timeA_nome2,
          # 'timeB_sigla': timeB_sigla,
          'timeB_nome': timeB_nome2,
          'resultado': resultado2,
          'fase': "Grupos - Rodada " + rodada + " - Grupo " + chr(grupo + 65),
          'ida ou volta': ida_ou_volta2,
          'torneio': "Sul-Americana",
      })

  # sulamericana_fase_grupos_resultado

  sulamericana_base_fase_grupos = pd.DataFrame(sulamericana_fase_grupos_resultado)

  display(sulamericana_base_fase_grupos.head())
  display(sulamericana_base_fase_grupos.tail())

,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,César Vallejo,LDU Quito,1-2,Grupos - Rodada 1 - Grupo A,Ida,Sul-Americana
1,06/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Magallanes,Botafogo,2-2,Grupos - Rodada 1 - Grupo A,Ida,Sul-Americana
2,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Danubio,Emelec,2-0,Grupos - Rodada 1 - Grupo B,Ida,Sul-Americana
3,06/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Huracán,Guaraní,4-1,Grupos - Rodada 1 - Grupo B,Ida,Sul-Americana
4,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,O. Petrolero,Estudiantes,0-1,Grupos - Rodada 1 - Grupo C,Ida,Sul-Americana


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
91,29/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Def y Justicia,Millonarios,3-1,Grupos - Rodada 6 - Grupo F,Volta,Sul-Americana
92,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Universitario,Gimn La Plata,1-0,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana
93,28/06/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Santa Fe,Goiás,1-2,Grupos - Rodada 6 - Grupo G,Volta,Sul-Americana
94,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,San Lorenzo,Estudiantes M.,4-1,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana
95,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Palestino,Fortaleza,1-2,Grupos - Rodada 6 - Grupo H,Volta,Sul-Americana


## Empilhamento de bases

In [57]:
if var_reler_dados == True:
  base_jogos_academia_apostas = pd.concat([
      libertadores_base_fase_grupos,
      sulamericana_base_fase_grupos,
      brasileirao_base_resultado,
      copabrasil_base_resultado
  ]).reset_index().drop("index", axis = 1)

  display(base_jogos_academia_apostas.head())
  display(base_jogos_academia_apostas.tail())

,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores


,data_jogo,link_jogo,timeA_nome,timeB_nome,resultado,fase,ida ou volta,torneio
682,13/07/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Palmeiras,São Paulo,vs,Quartas de Final,Volta,Copa do Brasil
683,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,América-MG,Corinthians,vs,Quartas de Final,Ida,Copa do Brasil
684,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,América-MG,vs,Quartas de Final,Volta,Copa do Brasil
685,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Athletico PR,vs,Quartas de Final,Ida,Copa do Brasil
686,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Flamengo,vs,Quartas de Final,Volta,Copa do Brasil


## Dados de cada jogo

In [140]:
# jogo_url = base_jogos_academia_apostas.loc[base_jogos_academia_apostas["torneio"] == "Libertadores", "link_jogo"].iloc[0]
# jogo_url = base_jogos_academia_apostas.loc[(base_jogos_academia_apostas["torneio"] == "Libertadores") * base_jogos_academia_apostas["timeB_nome"] == "Palmeiras", "link_jogo"].iloc[0]
# jogo_url = base_jogos_academia_apostas.loc[base_jogos_academia_apostas["torneio"] == "Brasileirão", "link_jogo"].iloc[0]
# jogo_url = base_jogos_academia_apostas.loc[base_jogos_academia_apostas["torneio"] == "Copa do Brasil", "link_jogo"].iloc[0]
# jogo_url = base_jogos_academia_apostas.loc[base_jogos_academia_apostas["torneio"] == "Sul-Americana", "link_jogo"].iloc[0]
# jogo_url = base_jogos_academia_apostas[base_jogos_academia_apostas["resultado"] == "vs"]["link_jogo"].iloc[0]
jogo_url = base_jogos_academia_apostas.iloc[624]["link_jogo"] # exemplo de jogo com penalti


jogo_response_url = requests.get(jogo_url)

jogo_soup = BeautifulSoup(jogo_response_url.content, 'html.parser')
# jogo_soup

# Se quiser exportar e ler os dados offline
# with open('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Jogo.html', "w") as file:
#   file.write(str(jogo_soup))


# jogo_detalhes = jogo_soup.\
#   find_all('table', class_='match-summary')[1].find_all("tr")[2]#.find_all("div", style_="float:right") # Dados do jogo
# len(jogo_detalhes)
# jogo_detalhes[0]
# jogo_detalhes


jogo_time_mandante = jogo_soup.find_all("div", "stats-game-head-teamname")[2].find_all("a")[0].text.strip()
jogo_time_visitante = jogo_soup.find_all("div", "stats-game-head-teamname")[2].find_all("a")[1].text.strip()


### EVENTOS DO JOGO, COMO GOLS, CARTÕES E SUBSTITUIÇÕES
jogo_eventos = []
# 0: primeiro tempo
# 1: segundo tempo
# 2: prorrogação, se houve

jogo_soup_eventos = jogo_soup.find_all('table', class_='match-summary')[0:3]
for tempos in jogo_soup_eventos:
  eventos_soup = tempos.find_all("tr")
  for evento in eventos_soup:

    minuto = evento.find_all("td", class_="match-sum-wd-minute")
    # display(minuto)

    if len(minuto) == 0:
      continue
    else:
      if minuto[0].text.strip() == "":
        minuto = minuto[1].text.strip()
        time_nome = jogo_time_mandante
        evento_jogo = evento.find_all("td", "match-sum-wd-symbol")[1].find("span").get("title")
      else:
        minuto = minuto[0].text.strip()
        time_nome = jogo_time_visitante
        evento_jogo = evento.find_all("td", "match-sum-wd-symbol")[0].find("span").get("title")

      # casos de VAR (não tem link do jogador)
      if evento.find("a") is None:
        jogador_nome = evento_jogo
        evento_jogo = evento.find("div").text.strip()
        jogador_link = ""
      else:
        jogador_nome = evento.find("a").text.strip()
        jogador_link = evento.find("a").get("href")


      jogo_eventos.append({
          'link_jogo': jogo_url,
          'jogador_nome': jogador_nome,
          'jogador_link': jogador_link,
          'evento': evento_jogo,
          'time_nome': time_nome,
          'minuto': minuto,
      })

      if evento_jogo == "Substitute in":
        jogo_eventos.append({
          'link_jogo': jogo_url,
          'jogador_nome': evento.find("div", class_="text-grey").find("a").text.strip(),
          'jogador_link': evento.find("div", class_="text-grey").find("a").get("href"),
          'evento': "Substitute out",
          'time_nome': time_nome,
          'minuto': minuto,
      })

# jogo_eventos


### DISPUTA DE PENALTIS
# 3: penaltis, se houve (PSM -> perdeu, PSG -> gol)

jogo_penaltis = []
# jogo_soup_penaltis_batedores_mandantes = jogo_soup.find_all('table', class_='match-summary')[3].find_all("td", class_="match-sum-wd-symbol pen6216 penpersonimg")
# display(jogo_soup_penaltis_batedores_mandantes)
jogo_soup_penaltis = jogo_soup.find_all('table', class_='match-summary')[3].find_all("td")#.find_all("td", class_="match-sum-wd-symbol pen6216 penpersonimg")
# display(jogo_soup_penaltis[1:5])

penalti_numero = ""
penalti_batedor_link = ""
penalti_batedor_nome = ""
penalti_resultado = ""

for dado_penaltis in jogo_soup_penaltis:#[5:10]:
  # display(dado_penaltis)

  if dado_penaltis.has_attr("class"):
    # print(dado_penaltis.attrs["class"][-1])

    if dado_penaltis.attrs["class"][-1] == "pennumber":
      penalti_numero = dado_penaltis.text.strip()
      # print(penalti_numero)
      if penalti_resultado == "":
        penalti_time = "Mandante"
      else:
        penalti_time = "Visitante"

    elif dado_penaltis.attrs["class"][-1] == "penperson":
      penalti_batedor_link = dado_penaltis.find("a").get("href")
      penalti_batedor_nome = dado_penaltis.find("a").text.strip()
      # print(penalti_batedor_nome)

    elif dado_penaltis.attrs["class"][-1] == "penresult":
      penalti_resultado = dado_penaltis.find("span").get("class")[1]
      if penalti_resultado == "PSM": #(PSM -> perdeu, PSG -> gol)
        penalti_resultado = "Não gol"
      else:
        penalti_resultado = "Gol"
      # print(penalti_resultado)

      if penalti_numero == "":
        penalti_time = "Visitante"
      else:
        penalti_time = "Mandante"

    else:
      continue

    # print(penalti_numero, penalti_batedor_nome, penalti_resultado)
    if (penalti_numero != "") * (penalti_batedor_nome != "") * (penalti_resultado != ""):
      jogo_penaltis.append({
          'link_jogo': jogo_url,
          'penalti_numero': penalti_numero,
          'penalti_batedor_link': penalti_batedor_link,
          'penalti_batedor_nome': penalti_batedor_nome,
          'penalti_resultado': penalti_resultado,
          'penalti_time': penalti_time,
      })
      penalti_numero = ""
      penalti_batedor_nome = ""
      penalti_resultado = ""

      # display(jogo_penaltis[-1])
      # display(print(""))

  else:
    continue

# jogo_penaltis
# jogo_penaltis_base = pd.DataFrame(jogo_penaltis)
# jogo_penaltis_base.sort_values("penalti_numero", ascending = True)


### ESCALAÇÃO COMPLETA DOS TIMES
# 4: escalação (vazio?)
# 5: escalação completa dos titulares (time mandante)
# 6: escalação completa dos titulares (time visitante)
# 7: substituições (vazio?)
# 8: escalação completa dos reservas (time mandante)
# 9: escalação completa dos reservas (time visitante)

jogo_escalacao = []
jogo_soup_escalacao = jogo_soup.find_all('table', class_='match-summary')[5:10]
for time_titular_reserva, escalacoes in enumerate(jogo_soup_escalacao):
  if time_titular_reserva == 0:
    titular_reserva = "Titular"
    time_nome = jogo_time_mandante
  elif time_titular_reserva == 1:
    titular_reserva = "Titular"
    time_nome = jogo_time_visitante
  elif time_titular_reserva == 2:
    continue
  elif time_titular_reserva == 3:
    titular_reserva = "Reserva"
    time_nome = jogo_time_mandante
  elif time_titular_reserva == 4:
    titular_reserva = "Reserva"
    time_nome = jogo_time_visitante

  # print(titular_reserva, time_nome)
  # display(escalacoes)

  escalacao = escalacoes.find_all("tr")
  for jogador in escalacao:
    # display(jogador)

    if time_nome == jogo_time_visitante:
      jogador_numero = jogador.find_all("td")[1].text.strip()
      jogador_posicao = jogador.find_all("td")[3].text.strip()
    else:
      jogador_numero = jogador.find_all("td")[2].text.strip()
      jogador_posicao = jogador.find_all("td")[0].text.strip()

    jogo_escalacao.append({
        'link_jogo': jogo_url,
        'jogador_nome': jogador.find("a").text.strip(),
        'jogador_link': jogador.find("a").get("href"),
        'jogador_pais': jogador.find("span").get("title"),
        'jogador_posicao': jogador_posicao,
        'jogador_numero': jogador_numero,
        'time_nome': time_nome,
        'titular_reserva': titular_reserva,
      })

# jogo_escalacao



### ESCALAÇÃO COMPLETA DOS TIMES
# 10: arbitragem

jogo_arbitragem = []
jogo_soup_arbitragem = jogo_soup.find_all('table', class_='match-summary')[10].find_all("td")#, align_="center")

# display(jogo_soup_arbitragem_funcoes)
for arbitros in jogo_soup_arbitragem:
  try:
    if arbitros.attrs["align"] == "center":
      arbitro_funcao = arbitros.text.strip()
    if arbitros.attrs["align"] == "left":
      arbitro_nome = arbitros.text.strip()
      arbitro_link = arbitros.find("a").get("href")
    else:
      continue
  except:
    continue

  # display(arbitro_funcao)

  if (arbitro_funcao == "") * (arbitro_nome == ""):
    continue
  else:
    jogo_arbitragem.append({
      'link_jogo': jogo_url,
      'arbitro_nome': arbitro_nome,
      'arbitro_link': arbitro_link,
      'arbitro_funcao': arbitro_funcao,
    })
    arbitro_funcao = ""
    arbitro_nome = ""

# jogo_arbitragem



### ESTATÍSTICAS MAIS DETALHADAS
# 11: estatísticas do jogo

jogo_estatisticas = []
jogo_soup_estatisticas = jogo_soup.find_all('table', class_='match-summary')[11].find_all("tr")
# display(len(jogo_soup_estatisticas))

for num_estatistica, estatistica in enumerate(jogo_soup_estatisticas):
  if num_estatistica == 0:
    continue
  else:
    dados_estatistica = estatistica.find_all("td")

    dado_estatistica_mandante = dados_estatistica[1].text.strip()
    nome_estatistica = estatistica.find_all("td")[2].text.strip()
    dado_estatistica_visitante = dados_estatistica[3].text.strip()

    jogo_estatisticas.append({
      'link_jogo': jogo_url,
      'nome_estatistica': nome_estatistica,
      'dado_estatistica_mandante': dado_estatistica_mandante,
      'dado_estatistica_visitante': dado_estatistica_visitante,
    })

# jogo_estatisticas




### ESTÁDIO DO JOGO
jogo_soup_estadio = jogo_soup.find_all('table', class_='match-stats-live-bg venue')[0].find_all("td")
jogo_estadio = []

jogo_estadio.append({
  'link_jogo': jogo_url,
  'nome_estadio': jogo_soup_estadio[3].text.strip(),
  'link_estadio': jogo_soup_estadio[0].find("a").get("href"),
  'cidade_estadio': jogo_soup_estadio[5].text.strip(),
  'capacidade_estadio': jogo_soup_estadio[7].text.strip(),
})

# jogo_estadio



### TÉCNICOS
# jogo_detalhes = jogo_soup.\
  # find_all('div', class_='pitch_position coach away tipsy-active') # Técnico visitante (muito buraco =( )
  # find_all('div', class_='pitch_position coach home tipsy-active') # Técnico mandante (muito buraco =( )

[{'link_jogo': 'https://www.academiadasapostasbrasil.com/stats/match/brasil/copa-do-brasil/atletico-go/volta-redonda/kznmO8JkNQxJd/live',
  'penalti_numero': '1º',
  'penalti_batedor_link': 'https://www.academiadasapostasbrasil.com/stats/person/brasil/shaylon/433191',
  'penalti_batedor_nome': 'Shaylon',
  'penalti_resultado': 'Não gol',
  'penalti_time': 'Mandante'},
 {'link_jogo': 'https://www.academiadasapostasbrasil.com/stats/match/brasil/copa-do-brasil/atletico-go/volta-redonda/kznmO8JkNQxJd/live',
  'penalti_numero': '2º',
  'penalti_batedor_link': 'https://www.academiadasapostasbrasil.com/stats/person/brasil/leanderson-da-silva-genesio/1nXYLz02Vm3RB',
  'penalti_batedor_nome': 'Leanderson da Silva Genesio',
  'penalti_resultado': 'Gol',
  'penalti_time': 'Visitante'},
 {'link_jogo': 'https://www.academiadasapostasbrasil.com/stats/match/brasil/copa-do-brasil/atletico-go/volta-redonda/kznmO8JkNQxJd/live',
  'penalti_numero': '4º',
  'penalti_batedor_link': 'https://www.academiadas

# Tratamento dos dados (site Academia Apostas Brasil)

In [59]:
base_jogos_academia_apostas_backup = base_jogos_academia_apostas.copy()
base_jogos_academia_apostas_tratada = base_jogos_academia_apostas.copy()

## Ajustar nomes colunas

In [60]:
base_jogos_academia_apostas.columns

Index(['data_jogo', 'link_jogo', 'timeA_nome', 'timeB_nome', 'resultado',
       'fase', 'ida ou volta', 'torneio'],
      dtype='object')

In [61]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.columns = [
      'Data',
      'Link para o jogo',
      # 'Time mandante (sigla)',
      'Time mandante (nome)',
      # 'Time visitante (sigla)',
      'Time visitante (nome)',
      'Resultado',
      'Fase do torneio',
      'Ida ou volta',
      'Torneio',
      # 'Local'
  ]

  display(base_jogos_academia_apostas_tratada.head())
  display(base_jogos_academia_apostas_tratada.tail())

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores


,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio
682,13/07/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Palmeiras,São Paulo,vs,Quartas de Final,Volta,Copa do Brasil
683,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,América-MG,Corinthians,vs,Quartas de Final,Ida,Copa do Brasil
684,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,América-MG,vs,Quartas de Final,Volta,Copa do Brasil
685,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Athletico PR,vs,Quartas de Final,Ida,Copa do Brasil
686,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Flamengo,vs,Quartas de Final,Volta,Copa do Brasil


## Tratar coluna de data e hora

In [62]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada["Data e Hora"] = pd.to_datetime(base_jogos_academia_apostas_tratada["Data"].str[:-1].str.strip(), format = "%d/%m/%Y %H:%M", errors='coerce')
  base_jogos_academia_apostas_tratada["Ano"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.year
  base_jogos_academia_apostas_tratada["Mês"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.month
  base_jogos_academia_apostas_tratada["Dia"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.day
  base_jogos_academia_apostas_tratada["Hora"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.hour
  base_jogos_academia_apostas_tratada["Minuto"] = base_jogos_academia_apostas_tratada["Data e Hora"].dt.minute

  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Mês"]).count().iloc[:, 0])
  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Hora"]).count().iloc[:, 0])
  # pd.DataFrame(base_jogos_cbf_tratada.groupby(["Torneio", "Minuto"]).count().iloc[:, 0])
  # base_jogos_cbf_tratada[base_jogos_cbf_tratada["Data e Hora"].isna()]

  display(base_jogos_academia_apostas_tratada[(~base_jogos_academia_apostas_tratada["Data e Hora"].isna()) * (base_jogos_academia_apostas_tratada["Torneio"] == "Libertadores")].head())
  display(base_jogos_academia_apostas_tratada[(~base_jogos_academia_apostas_tratada["Data e Hora"].isna()) * (base_jogos_academia_apostas_tratada["Torneio"] == "Libertadores")].tail())

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,Mês,Dia,Hora,Minuto
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 19:00:00,2023,4,5,19,0
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 21:00:00,2023,4,5,21,0
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 21:00:00,2023,4,4,21,0
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 23:00:00,2023,4,4,23,0
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores,2023-04-05 21:00:00,2023,4,5,21,0


,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,Mês,Dia,Hora,Minuto
91,29/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Colo Colo,Pereira,0-0,Grupos - Rodada 6 - Grupo F,Volta,Libertadores,2023-06-29 19:00:00,2023,6,29,19,0
92,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Alianza Lima,3-0,Grupos - Rodada 6 - Grupo G,Volta,Libertadores,2023-06-27 19:00:00,2023,6,27,19,0
93,27/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Libertad,Atlético-MG,1-1,Grupos - Rodada 6 - Grupo G,Volta,Libertadores,2023-06-27 19:00:00,2023,6,27,19,0
94,27/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Atl. Nacional,Patronato,0-1,Grupos - Rodada 6 - Grupo H,Volta,Libertadores,2023-06-27 21:00:00,2023,6,27,21,0
95,27/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Olimpia,Melgar,4-1,Grupos - Rodada 6 - Grupo H,Volta,Libertadores,2023-06-27 21:00:00,2023,6,27,21,0


## Separação dos resultados de penaltis

In [63]:
# if var_reler_dados == True:
#   display(base_jogos_academia_apostas_tratada[
#       (base_jogos_academia_apostas_tratada["Fase do torneio"] == "3ª Fase") \
#       * (base_jogos_academia_apostas_tratada["Ida ou volta"] == "Volta")].iloc[0:5])

In [64]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada["Penaltis_temp"] =  base_jogos_academia_apostas_tratada["Resultado"].str.find("p.g.")
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Penaltis_temp"] < 0, "Houve penaltis?"] = (False)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Penaltis_temp"] >= 0, "Houve penaltis?"] = (True)

  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]][["Resultado", "Penaltis_temp", "Houve penaltis?"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False][["Resultado", "Penaltis_temp", "Houve penaltis?"]].head())

  base_jogos_academia_apostas_tratada = base_jogos_academia_apostas_tratada.drop("Penaltis_temp", axis = 1)

,Resultado,Penaltis_temp,Houve penaltis?
624,1-1e.t.(1-1)p.g.(4-5),12,True
630,0-0e.t.(0-0)p.g.(5-6),12,True
636,2-1e.t.(2-1)p.g.(6-7),12,True
648,2-1e.t.(2-1)p.g.(4-2),12,True
652,2-0e.t.(2-0)p.g.(5-4),12,True


,Resultado,Penaltis_temp,Houve penaltis?
0,2-1,-1,False
1,0-2,-1,False
2,1-1,-1,False
3,1-2,-1,False
4,2-1,-1,False


In [65]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Penaltis mandante"] = \
      base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]]["Resultado"] \
      .str.split("p.g.(", regex = False).str[1] \
      .str.split("-", regex = False).str[0].astype(int)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False, "Penaltis mandante"] = np.nan

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Penaltis visitante"] = \
      base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]]["Resultado"] \
      .str.split("p.g.(", regex = False).str[1] \
      .str.split("-", regex = False).str[1] \
      .str.split(")", regex = False).str[0].astype(int)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False, "Penaltis visitante"] = np.nan

  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]][["Resultado", "Penaltis mandante", "Penaltis visitante"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False][["Resultado", "Penaltis mandante", "Penaltis visitante"]].head())

,Resultado,Penaltis mandante,Penaltis visitante
624,1-1e.t.(1-1)p.g.(4-5),4.0,5.0
630,0-0e.t.(0-0)p.g.(5-6),5.0,6.0
636,2-1e.t.(2-1)p.g.(6-7),6.0,7.0
648,2-1e.t.(2-1)p.g.(4-2),4.0,2.0
652,2-0e.t.(2-0)p.g.(5-4),5.0,4.0


,Resultado,Penaltis mandante,Penaltis visitante
0,2-1,NaN,NaN
1,0-2,NaN,NaN
2,1-1,NaN,NaN
3,1-2,NaN,NaN
4,2-1,NaN,NaN


In [66]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Resultado (dos penaltis)"] = \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Penaltis mandante"].astype(str).str[0] \
      + " x " + \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"], "Penaltis visitante"].astype(str).str[0]

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Houve penaltis?"] == False, "Resultado (dos penaltis)"] = np.nan

  display(base_jogos_academia_apostas_tratada[["Resultado", "Resultado (dos penaltis)"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]][["Resultado", "Resultado (dos penaltis)"]].head())

,Resultado,Resultado (dos penaltis)
0,2-1,NaN
1,0-2,NaN
2,1-1,NaN
3,1-2,NaN
4,2-1,NaN


,Resultado,Resultado (dos penaltis)
624,1-1e.t.(1-1)p.g.(4-5),4 x 5
630,0-0e.t.(0-0)p.g.(5-6),5 x 6
636,2-1e.t.(2-1)p.g.(6-7),6 x 7
648,2-1e.t.(2-1)p.g.(4-2),4 x 2
652,2-0e.t.(2-0)p.g.(5-4),5 x 4


## Mais detalhes do resultado

In [67]:
# base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Resultado"] == "Bahia"]
base_jogos_academia_apostas_tratada.iloc[679:689]

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,Mês,Dia,Hora,Minuto,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis)
679,04/07/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Bahia,Grêmio,vs,Quartas de Final,Ida,Copa do Brasil,2023-07-04 21:00:00,2023,7,4,21,0,False,NaN,NaN,NaN
680,12/07/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Grêmio,Bahia,vs,Quartas de Final,Volta,Copa do Brasil,2023-07-12 19:00:00,2023,7,12,19,0,False,NaN,NaN,NaN
681,05/07/2023 19:30h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Palmeiras,vs,Quartas de Final,Ida,Copa do Brasil,2023-07-05 19:30:00,2023,7,5,19,30,False,NaN,NaN,NaN
682,13/07/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Palmeiras,São Paulo,vs,Quartas de Final,Volta,Copa do Brasil,2023-07-13 20:00:00,2023,7,13,20,0,False,NaN,NaN,NaN
683,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,América-MG,Corinthians,vs,Quartas de Final,Ida,Copa do Brasil,2023-07-05 21:30:00,2023,7,5,21,30,False,NaN,NaN,NaN
684,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,América-MG,vs,Quartas de Final,Volta,Copa do Brasil,2023-07-12 21:30:00,2023,7,12,21,30,False,NaN,NaN,NaN
685,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Athletico PR,vs,Quartas de Final,Ida,Copa do Brasil,2023-07-05 21:30:00,2023,7,5,21,30,False,NaN,NaN,NaN
686,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Flamengo,vs,Quartas de Final,Volta,Copa do Brasil,2023-07-12 21:30:00,2023,7,12,21,30,False,NaN,NaN,NaN


In [68]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] == "vs", "Resultado"] = "Jogo ainda não ocorreu"

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"] = \
    base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado"] \
    .str.split("e.t.", regex = False).str[0] \
    .str.split("-", regex = False).str[0].astype(int)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu", "Gols mandante"] = np.nan

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"] = \
    base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado"] \
    .str.split("e.t.", regex = False).str[0] \
    .str.split("-", regex = False).str[1].astype(int)
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu", "Gols visitante"] = np.nan


  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu"][["Resultado", "Gols mandante", "Gols visitante"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu"][["Resultado", "Gols mandante", "Gols visitante"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu"][["Resultado", "Gols mandante", "Gols visitante"]].tail())

,Resultado,Gols mandante,Gols visitante
321,Jogo ainda não ocorreu,NaN,NaN
322,Jogo ainda não ocorreu,NaN,NaN
323,Jogo ainda não ocorreu,NaN,NaN
324,Jogo ainda não ocorreu,NaN,NaN
325,Jogo ainda não ocorreu,NaN,NaN


,Resultado,Gols mandante,Gols visitante
0,2-1,2.0,1.0
1,0-2,0.0,2.0
2,1-1,1.0,1.0
3,1-2,1.0,2.0
4,2-1,2.0,1.0


,Resultado,Gols mandante,Gols visitante
674,1-0e.t.(1-0)p.g.(2-4),1.0,0.0
675,2-0,2.0,0.0
676,3-1e.t.(3-1)p.g.(4-5),3.0,1.0
677,2-0,2.0,0.0
678,2-0e.t.(2-0)p.g.(3-1),2.0,0.0


In [69]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Resultado (sem penaltis)"] = \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols mandante"].astype(str).str[0] \
      + " x " + \
      base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] != "Jogo ainda não ocorreu", "Gols visitante"].astype(str).str[0]

  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu", "Resultado (sem penaltis)"] = "Jogo ainda não ocorreu"

  display(base_jogos_academia_apostas_tratada[["Resultado", "Resultado (sem penaltis)"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Houve penaltis?"]][["Resultado", "Resultado (sem penaltis)"]].head())
  display(base_jogos_academia_apostas_tratada[base_jogos_academia_apostas_tratada["Resultado"] == "Jogo ainda não ocorreu"][["Resultado", "Resultado (sem penaltis)"]].head())

,Resultado,Resultado (sem penaltis)
0,2-1,2 x 1
1,0-2,0 x 2
2,1-1,1 x 1
3,1-2,1 x 2
4,2-1,2 x 1


,Resultado,Resultado (sem penaltis)
624,1-1e.t.(1-1)p.g.(4-5),1 x 1
630,0-0e.t.(0-0)p.g.(5-6),0 x 0
636,2-1e.t.(2-1)p.g.(6-7),2 x 1
648,2-1e.t.(2-1)p.g.(4-2),2 x 1
652,2-0e.t.(2-0)p.g.(5-4),2 x 0


,Resultado,Resultado (sem penaltis)
321,Jogo ainda não ocorreu,Jogo ainda não ocorreu
322,Jogo ainda não ocorreu,Jogo ainda não ocorreu
323,Jogo ainda não ocorreu,Jogo ainda não ocorreu
324,Jogo ainda não ocorreu,Jogo ainda não ocorreu
325,Jogo ainda não ocorreu,Jogo ainda não ocorreu


In [70]:
if var_reler_dados == True:
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Gols mandante"] > base_jogos_academia_apostas_tratada["Gols visitante"], "Resultado jogo"] = "Mandante"
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Gols mandante"] < base_jogos_academia_apostas_tratada["Gols visitante"], "Resultado jogo"] = "Visitante"
  base_jogos_academia_apostas_tratada.loc[base_jogos_academia_apostas_tratada["Gols mandante"] == base_jogos_academia_apostas_tratada["Gols visitante"], "Resultado jogo"] = "Empate"

  display(base_jogos_academia_apostas_tratada.head())
  display(base_jogos_academia_apostas_tratada.tail())

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Hora,Minuto,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis),Gols mandante,Gols visitante,Resultado (sem penaltis),Resultado jogo
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 19:00:00,2023,...,19,0,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 21:00:00,2023,...,21,0,False,NaN,NaN,NaN,0.0,2.0,0 x 2,Visitante
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 21:00:00,2023,...,21,0,False,NaN,NaN,NaN,1.0,1.0,1 x 1,Empate
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 23:00:00,2023,...,23,0,False,NaN,NaN,NaN,1.0,2.0,1 x 2,Visitante
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores,2023-04-05 21:00:00,2023,...,21,0,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante


,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Hora,Minuto,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis),Gols mandante,Gols visitante,Resultado (sem penaltis),Resultado jogo
682,13/07/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Palmeiras,São Paulo,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,2023-07-13 20:00:00,2023,...,20,0,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN
683,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,América-MG,Corinthians,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,2023-07-05 21:30:00,2023,...,21,30,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN
684,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,América-MG,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,2023-07-12 21:30:00,2023,...,21,30,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN
685,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Athletico PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,2023-07-05 21:30:00,2023,...,21,30,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN
686,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Flamengo,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,2023-07-12 21:30:00,2023,...,21,30,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN


## Ajustar base de jogadores/escalações

## Ajustar base de Árbitros

## Ajustar base de Estatísticas

## Exportando dados

In [71]:
display(base_jogos_academia_apostas_tratada.columns)
display(base_jogos_academia_apostas_tratada.head())

Index(['Data', 'Link para o jogo', 'Time mandante (nome)',
       'Time visitante (nome)', 'Resultado', 'Fase do torneio', 'Ida ou volta',
       'Torneio', 'Data e Hora', 'Ano', 'Mês', 'Dia', 'Hora', 'Minuto',
       'Houve penaltis?', 'Penaltis mandante', 'Penaltis visitante',
       'Resultado (dos penaltis)', 'Gols mandante', 'Gols visitante',
       'Resultado (sem penaltis)', 'Resultado jogo'],
      dtype='object')

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Hora,Minuto,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis),Gols mandante,Gols visitante,Resultado (sem penaltis),Resultado jogo
0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 19:00:00,2023,...,19,0,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante
1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 21:00:00,2023,...,21,0,False,NaN,NaN,NaN,0.0,2.0,0 x 2,Visitante
2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 21:00:00,2023,...,21,0,False,NaN,NaN,NaN,1.0,1.0,1 x 1,Empate
3,04/04/2023 23:00h,https://www.academiadasapostasbrasil.com/stats...,Metropolitanos,Nacional,1-2,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 23:00:00,2023,...,23,0,False,NaN,NaN,NaN,1.0,2.0,1 x 2,Visitante
4,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cerro Porteño,Barcelona,2-1,Grupos - Rodada 1 - Grupo C,Ida,Libertadores,2023-04-05 21:00:00,2023,...,21,0,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante


In [72]:
base_jogos_academia_apostas_tratada.to_excel('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Base Jogos Tratada.xlsx')
# base_jogos_academia_apostas_tratada.to_excel("Tabela Brasileirão.xlsx")

In [73]:
base_jogos_tratada = base_jogos_academia_apostas_tratada.copy()

In [74]:
base_jogos_tratada[(base_jogos_tratada["Torneio"] == "Brasileirão") * (base_jogos_tratada["Fase do torneio"] == "Rodada 13")]

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Hora,Minuto,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis),Gols mandante,Gols visitante,Resultado (sem penaltis),Resultado jogo
312,01/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Fluminense,1-0,Rodada 13,Ida,Brasileirão,2023-07-01 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,1.0,0.0,1 x 0,Mandante
313,01/07/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Bahia,Grêmio,1-2,Rodada 13,Ida,Brasileirão,2023-07-01 18:30:00,2023,...,18,30,False,NaN,NaN,NaN,1.0,2.0,1 x 2,Visitante
314,01/07/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Fortaleza,2-0,Rodada 13,Ida,Brasileirão,2023-07-01 18:30:00,2023,...,18,30,False,NaN,NaN,NaN,2.0,0.0,2 x 0,Mandante
315,01/07/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Internacional,Cruzeiro,0-0,Rodada 13,Ida,Brasileirão,2023-07-01 21:00:00,2023,...,21,0,False,NaN,NaN,NaN,0.0,0.0,0 x 0,Empate
316,02/07/2023 11:00h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,RB Bragantino,0-1,Rodada 13,Ida,Brasileirão,2023-07-02 11:00:00,2023,...,11,0,False,NaN,NaN,NaN,0.0,1.0,0 x 1,Visitante
317,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Botafogo,Vasco,2-0,Rodada 13,Ida,Brasileirão,2023-07-02 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,2.0,0.0,2 x 0,Mandante
318,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Palmeiras,2-2,Rodada 13,Ida,Brasileirão,2023-07-02 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,2.0,2.0,2 x 2,Empate
319,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Atlético-MG,América-MG,2-2,Rodada 13,Ida,Brasileirão,2023-07-02 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,2.0,2.0,2 x 2,Empate
320,02/07/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Cuiabá,Santos,3-0,Rodada 13,Ida,Brasileirão,2023-07-02 18:30:00,2023,...,18,30,False,NaN,NaN,NaN,3.0,0.0,3 x 0,Mandante
321,03/07/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Goiás,Coritiba,Jogo ainda não ocorreu,Rodada 13,Ida,Brasileirão,2023-07-03 20:00:00,2023,...,20,0,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN


# Base "duplicada", para ver todos os jogos do time

In [75]:
base_jogos_tratada_duplicada = base_jogos_tratada.copy()
base_jogos_tratada_duplicada_temp = base_jogos_tratada.copy()

base_jogos_tratada_duplicada["Time (nome)"] = base_jogos_tratada_duplicada["Time mandante (nome)"]

base_jogos_tratada_duplicada_temp["Time (nome)"] = base_jogos_tratada_duplicada["Time visitante (nome)"]

base_jogos_tratada_duplicada = pd.concat([base_jogos_tratada_duplicada, base_jogos_tratada_duplicada_temp])

# base_jogos_tratada_duplicada
base_jogos_tratada_duplicada[
    # (base_jogos_tratada_duplicada["Time (nome)"] == "Palmeiras") \
    # (base_jogos_tratada_duplicada["Time (nome)"] == "Flamengo") \
    # (base_jogos_tratada_duplicada["Time (nome)"] == "São Paulo") \
    (base_jogos_tratada_duplicada["Time (nome)"] == "Grêmio") \
    * (base_jogos_tratada_duplicada["Resultado"] != "Jogo ainda não ocorreu") \
    # * (base_jogos_tratada_duplicada["Torneio"] == "Copa do Brasil")
    ].sort_values("Data e Hora", ascending = False).head(10)

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Minuto,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis),Gols mandante,Gols visitante,Resultado (sem penaltis),Resultado jogo,Time (nome)
313,01/07/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Bahia,Grêmio,1-2,Rodada 13,Ida,Brasileirão,2023-07-01 18:30:00,2023,...,30,False,NaN,NaN,NaN,1.0,2.0,1 x 2,Visitante,Grêmio
307,25/06/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Grêmio,Coritiba,5-1,Rodada 12,Ida,Brasileirão,2023-06-25 16:00:00,2023,...,0,False,NaN,NaN,NaN,5.0,1.0,5 x 1,Mandante,Grêmio
297,22/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Grêmio,América-MG,3-1,Rodada 11,Ida,Brasileirão,2023-06-22 19:00:00,2023,...,0,False,NaN,NaN,NaN,3.0,1.0,3 x 1,Mandante,Grêmio
290,11/06/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Grêmio,3-0,Rodada 10,Ida,Brasileirão,2023-06-11 18:30:00,2023,...,30,False,NaN,NaN,NaN,3.0,0.0,3 x 0,Mandante,Grêmio
278,04/06/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Grêmio,São Paulo,2-1,Rodada 9,Ida,Brasileirão,2023-06-04 16:00:00,2023,...,0,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante,Grêmio
670,31/05/2023 20:00h,https://www.academiadasapostasbrasil.com/stats...,Cruzeiro,Grêmio,0-1,Oitavas de Final,Volta,Copa do Brasil,2023-05-31 20:00:00,2023,...,0,False,NaN,NaN,NaN,0.0,1.0,0 x 1,Visitante,Grêmio
262,27/05/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Grêmio,1-2,Rodada 8,Ida,Brasileirão,2023-05-27 16:00:00,2023,...,0,False,NaN,NaN,NaN,1.0,2.0,1 x 2,Visitante,Grêmio
260,21/05/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Grêmio,Internacional,3-1,Rodada 7,Ida,Brasileirão,2023-05-21 18:30:00,2023,...,30,False,NaN,NaN,NaN,3.0,1.0,3 x 1,Mandante,Grêmio
669,17/05/2023 19:30h,https://www.academiadasapostasbrasil.com/stats...,Grêmio,Cruzeiro,1-1,Oitavas de Final,Ida,Copa do Brasil,2023-05-17 19:30:00,2023,...,30,False,NaN,NaN,NaN,1.0,1.0,1 x 1,Empate,Grêmio
248,14/05/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Grêmio,Fortaleza,0-0,Rodada 6,Ida,Brasileirão,2023-05-14 16:00:00,2023,...,0,False,NaN,NaN,NaN,0.0,0.0,0 x 0,Empate,Grêmio


In [76]:
base_jogos_tratada_duplicada.loc[
    (base_jogos_tratada_duplicada["Resultado jogo"] == "Mandante") * \
    (base_jogos_tratada_duplicada["Time mandante (nome)"] == base_jogos_tratada_duplicada["Time (nome)"]),
    "Resultado do Time"] = "Vitória"
base_jogos_tratada_duplicada.loc[
    (base_jogos_tratada_duplicada["Resultado jogo"] == "Visitante") * \
    (base_jogos_tratada_duplicada["Time visitante (nome)"] == base_jogos_tratada_duplicada["Time (nome)"]),
    "Resultado do Time"] = "Vitória"

base_jogos_tratada_duplicada.loc[
    (base_jogos_tratada_duplicada["Resultado jogo"] == "Empate"),
    "Resultado do Time"] = "Empate"

base_jogos_tratada_duplicada.loc[
    (base_jogos_tratada_duplicada["Resultado do Time"].isna() \
    * (base_jogos_tratada_duplicada["Resultado"] != "Jogo ainda não ocorreu")),
    "Resultado do Time"] = "Derrota"

# base_jogos_tratada_duplicada
base_jogos_tratada_duplicada[
    (base_jogos_tratada_duplicada["Time (nome)"] == "São Paulo") \
    # (base_jogos_tratada_duplicada["Time (nome)"] == "Palmeiras") \
    # (base_jogos_tratada_duplicada["Time (nome)"] == "Flamengo") \
    * (base_jogos_tratada_duplicada["Resultado"] != "Jogo ainda não ocorreu") \
    ].sort_values("Data e Hora", ascending = False).head(10)

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis),Gols mandante,Gols visitante,Resultado (sem penaltis),Resultado jogo,Time (nome),Resultado do Time
312,01/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Fluminense,1-0,Rodada 13,Ida,Brasileirão,2023-07-01 16:00:00,2023,...,False,NaN,NaN,NaN,1.0,0.0,1 x 0,Mandante,São Paulo,Vitória
183,27/06/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Tigre,2-0,Grupos - Rodada 6 - Grupo D,Volta,Sul-Americana,2023-06-27 21:30:00,2023,...,False,NaN,NaN,NaN,2.0,0.0,2 x 0,Mandante,São Paulo,Vitória
305,24/06/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Cruzeiro,São Paulo,1-0,Rodada 12,Ida,Brasileirão,2023-06-24 21:00:00,2023,...,False,NaN,NaN,NaN,1.0,0.0,1 x 0,Mandante,São Paulo,Derrota
292,21/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Athletico PR,2-1,Rodada 11,Ida,Brasileirão,2023-06-21 19:00:00,2023,...,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante,São Paulo,Vitória
288,11/06/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Palmeiras,0-2,Rodada 10,Ida,Brasileirão,2023-06-11 16:00:00,2023,...,False,NaN,NaN,NaN,0.0,2.0,0 x 2,Visitante,São Paulo,Derrota
167,08/06/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Dep. Tolima,5-0,Grupos - Rodada 5 - Grupo D,Volta,Sul-Americana,2023-06-08 19:00:00,2023,...,False,NaN,NaN,NaN,5.0,0.0,5 x 0,Mandante,São Paulo,Vitória
278,04/06/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Grêmio,São Paulo,2-1,Rodada 9,Ida,Brasileirão,2023-06-04 16:00:00,2023,...,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante,São Paulo,Derrota
672,01/06/2023 19:30h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Sport,1-3e.t.(1-3)p.g.(5-3),Oitavas de Final,Volta,Copa do Brasil,2023-06-01 19:30:00,2023,...,True,5.0,3.0,5 x 3,1.0,3.0,1 x 3,Visitante,São Paulo,Derrota
266,27/05/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,São Paulo,Goiás,2-1,Rodada 8,Ida,Brasileirão,2023-05-27 21:00:00,2023,...,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante,São Paulo,Vitória
150,23/05/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Puerto Cabello,São Paulo,0-2,Grupos - Rodada 4 - Grupo D,Volta,Sul-Americana,2023-05-23 21:30:00,2023,...,False,NaN,NaN,NaN,0.0,2.0,0 x 2,Visitante,São Paulo,Vitória


In [77]:
base_jogos_tratada_duplicada = base_jogos_tratada_duplicada.sort_index().reset_index().rename(columns = {"index": "index base tratada"})
base_jogos_tratada_duplicada

,index base tratada,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,...,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis),Gols mandante,Gols visitante,Resultado (sem penaltis),Resultado jogo,Time (nome),Resultado do Time
0,0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 19:00:00,...,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante,Aucas,Vitória
1,0,05/04/2023 19:00h,https://www.academiadasapostasbrasil.com/stats...,Aucas,Flamengo,2-1,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 19:00:00,...,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante,Flamengo,Derrota
2,1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 21:00:00,...,False,NaN,NaN,NaN,0.0,2.0,0 x 2,Visitante,Racing Club,Vitória
3,1,05/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Ñublense,Racing Club,0-2,Grupos - Rodada 1 - Grupo A,Ida,Libertadores,2023-04-05 21:00:00,...,False,NaN,NaN,NaN,0.0,2.0,0 x 2,Visitante,Ñublense,Derrota
4,2,04/04/2023 21:00h,https://www.academiadasapostasbrasil.com/stats...,Medellín,Internacional,1-1,Grupos - Rodada 1 - Grupo B,Ida,Libertadores,2023-04-04 21:00:00,...,False,NaN,NaN,NaN,1.0,1.0,1 x 1,Empate,Medellín,Empate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1369,684,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,América-MG,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,2023-07-12 21:30:00,...,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN,Corinthians,NaN
1370,685,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Athletico PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,2023-07-05 21:30:00,...,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN,Athletico PR,NaN
1371,685,05/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Flamengo,Athletico PR,Jogo ainda não ocorreu,Quartas de Final,Ida,Copa do Brasil,2023-07-05 21:30:00,...,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN,Flamengo,NaN
1372,686,12/07/2023 21:30h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Flamengo,Jogo ainda não ocorreu,Quartas de Final,Volta,Copa do Brasil,2023-07-12 21:30:00,...,False,NaN,NaN,NaN,NaN,NaN,Jogo ainda não ocorreu,NaN,Athletico PR,NaN


## Exportando dados

In [78]:
base_jogos_tratada_duplicada.to_excel('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Base Jogos Tratada Duplicada.xlsx')
# base_jogos_tratada_duplicada.to_excel("Tabela Brasileirão Duplicada.xlsx")

# Tabela de pontos Brasileirão

In [79]:
tabela_brasileirao = base_jogos_tratada[(base_jogos_tratada["Torneio"] == "Brasileirão") * (base_jogos_tratada["Resultado"] != "Jogo ainda não ocorreu")].copy()

tabela_brasileirao

,Data,Link para o jogo,Time mandante (nome),Time visitante (nome),Resultado,Fase do torneio,Ida ou volta,Torneio,Data e Hora,Ano,...,Hora,Minuto,Houve penaltis?,Penaltis mandante,Penaltis visitante,Resultado (dos penaltis),Gols mandante,Gols visitante,Resultado (sem penaltis),Resultado jogo
192,15/04/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Palmeiras,Cuiabá,2-1,Rodada 1,Ida,Brasileirão,2023-04-15 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante
193,15/04/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,América-MG,Fluminense,0-3,Rodada 1,Ida,Brasileirão,2023-04-15 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,0.0,3.0,0 x 3,Visitante
194,15/04/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Botafogo,São Paulo,2-1,Rodada 1,Ida,Brasileirão,2023-04-15 18:30:00,2023,...,18,30,False,NaN,NaN,NaN,2.0,1.0,2 x 1,Mandante
195,15/04/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Fortaleza,Internacional,1-1,Rodada 1,Ida,Brasileirão,2023-04-15 18:30:00,2023,...,18,30,False,NaN,NaN,NaN,1.0,1.0,1 x 1,Empate
196,15/04/2023 18:30h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Goiás,2-0,Rodada 1,Ida,Brasileirão,2023-04-15 18:30:00,2023,...,18,30,False,NaN,NaN,NaN,2.0,0.0,2 x 0,Mandante
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,02/07/2023 11:00h,https://www.academiadasapostasbrasil.com/stats...,Corinthians,RB Bragantino,0-1,Rodada 13,Ida,Brasileirão,2023-07-02 11:00:00,2023,...,11,0,False,NaN,NaN,NaN,0.0,1.0,0 x 1,Visitante
317,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Botafogo,Vasco,2-0,Rodada 13,Ida,Brasileirão,2023-07-02 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,2.0,0.0,2 x 0,Mandante
318,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Athletico PR,Palmeiras,2-2,Rodada 13,Ida,Brasileirão,2023-07-02 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,2.0,2.0,2 x 2,Empate
319,02/07/2023 16:00h,https://www.academiadasapostasbrasil.com/stats...,Atlético-MG,América-MG,2-2,Rodada 13,Ida,Brasileirão,2023-07-02 16:00:00,2023,...,16,0,False,NaN,NaN,NaN,2.0,2.0,2 x 2,Empate


In [80]:
# Estatísticas como mandante
tabela_brasileirao_mandante_temp = tabela_brasileirao.groupby(["Time mandante (nome)", "Resultado jogo"]).count()[["Link para o jogo"]]
tabela_brasileirao_mandante_temp = tabela_brasileirao_mandante_temp.join(
        tabela_brasileirao[["Time mandante (nome)", "Resultado jogo", "Gols mandante", "Gols visitante"]].groupby(["Time mandante (nome)", "Resultado jogo"]).sum()[["Gols mandante", "Gols visitante"]]
    ).reset_index()
tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Empate", "Pontos"] = \
    1 * tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Empate", "Link para o jogo"]
tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Mandante", "Pontos"] = \
    3 * tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Mandante", "Link para o jogo"]
tabela_brasileirao_mandante_temp.loc[tabela_brasileirao_mandante_temp["Resultado jogo"] == "Visitante", "Pontos"] = 0

tabela_brasileirao_mandante_temp = tabela_brasileirao_mandante_temp.rename(
    {"Link para o jogo": "Qtd jogos (como mandante)",
     "Time mandante (nome)": "Time",
     "Pontos": "Pontos (como mandante)",
     "Gols mandante": "Gols pró (como mandante)",
     "Gols visitante": "Gols contra (como mandante)",}, axis = 1)

tabela_brasileirao_mandante_temp = tabela_brasileirao_mandante_temp.set_index(["Time", "Resultado jogo"])
tabela_brasileirao_mandante_temp

# Estatísticas como visitante
tabela_brasileirao_visitante_temp = tabela_brasileirao.groupby(["Time visitante (nome)", "Resultado jogo"]).count()[["Link para o jogo"]]
tabela_brasileirao_visitante_temp = tabela_brasileirao_visitante_temp.join(
        tabela_brasileirao[["Time visitante (nome)", "Resultado jogo", "Gols mandante", "Gols visitante"]].groupby(["Time visitante (nome)", "Resultado jogo"]).sum()[["Gols mandante", "Gols visitante"]]
    ).reset_index()
tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Empate", "Pontos"] = \
    1 * tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Empate", "Link para o jogo"]
tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Visitante", "Pontos"] = \
    3 * tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Visitante", "Link para o jogo"]
tabela_brasileirao_visitante_temp.loc[tabela_brasileirao_visitante_temp["Resultado jogo"] == "Mandante", "Pontos"] = 0

tabela_brasileirao_visitante_temp = tabela_brasileirao_visitante_temp.rename(
    {"Link para o jogo": "Qtd jogos (como visitante)",
     "Time visitante (nome)": "Time",
     "Pontos": "Pontos (como visitante)",
     "Gols visitante": "Gols pró (como visitante)",
     "Gols mandante": "Gols contra (como visitante)",}, axis = 1)

tabela_brasileirao_visitante_temp = tabela_brasileirao_visitante_temp.set_index(["Time", "Resultado jogo"])
tabela_brasileirao_visitante_temp


# Totais
tabela_brasileirao_temp = tabela_brasileirao_visitante_temp.join(tabela_brasileirao_mandante_temp, how = "outer").fillna(0).reset_index()

# Pontos totais
tabela_brasileirao_temp["Pontos (total)"] = tabela_brasileirao_temp["Pontos (como visitante)"] + tabela_brasileirao_temp["Pontos (como mandante)"]

# Jogos totais
tabela_brasileirao_temp["Qtd jogos (total)"] = tabela_brasileirao_temp["Qtd jogos (como visitante)"] + tabela_brasileirao_temp["Qtd jogos (como mandante)"]

# Empates como mandante, como visitante e totais
tabela_brasileirao_temp["Qtd empates (como mandante)"] = \
    tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Empate", "Qtd jogos (como mandante)"]
tabela_brasileirao_temp["Qtd empates (como visitante)"] = \
    tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Empate", "Qtd jogos (como visitante)"]
tabela_brasileirao_temp = tabela_brasileirao_temp.fillna(0)
tabela_brasileirao_temp["Qtd empates (total)"] = tabela_brasileirao_temp["Qtd empates (como mandante)"] + tabela_brasileirao_temp["Qtd empates (como visitante)"]


# Derrotas totais
tabela_brasileirao_temp["Qtd derrotas (como visitante)"] = \
    tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Mandante", "Qtd jogos (como visitante)"]
tabela_brasileirao_temp["Qtd derrotas (como mandante)"] = \
    tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Visitante", "Qtd jogos (como mandante)"]
tabela_brasileirao_temp = tabela_brasileirao_temp.fillna(0)
tabela_brasileirao_temp["Qtd derrotas (total)"] = tabela_brasileirao_temp["Qtd derrotas (como mandante)"] + tabela_brasileirao_temp["Qtd derrotas (como visitante)"]

# Vitórias totais
tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Mandante", "Qtd vitórias (total)"] = tabela_brasileirao_temp["Qtd jogos (como mandante)"]
tabela_brasileirao_temp.loc[tabela_brasileirao_temp["Resultado jogo"] == "Visitante", "Qtd vitórias (total)"] = tabela_brasileirao_temp["Qtd jogos (como visitante)"]
tabela_brasileirao_temp = tabela_brasileirao_temp.fillna(0)

# Gols totais
tabela_brasileirao_temp["Gols pró"] = tabela_brasileirao_temp["Gols pró (como visitante)"] + tabela_brasileirao_temp["Gols pró (como mandante)"]
tabela_brasileirao_temp["Gols contra"] = tabela_brasileirao_temp["Gols contra (como visitante)"] + tabela_brasileirao_temp["Gols contra (como mandante)"]

# Tabela final
tabela_brasileirao_final = tabela_brasileirao_temp.groupby("Time").sum() \
    [["Pontos (total)", "Qtd jogos (total)", "Qtd vitórias (total)", "Qtd empates (total)", "Qtd derrotas (total)", "Gols pró", "Gols contra"]]
tabela_brasileirao_final["Saldo de gols"] = tabela_brasileirao_final["Gols pró"] - tabela_brasileirao_final["Gols contra"]
tabela_brasileirao_final["% aproveitamento"] = tabela_brasileirao_final["Pontos (total)"]/(tabela_brasileirao_final["Qtd jogos (total)"]*3)
tabela_brasileirao_final = tabela_brasileirao_final.sort_values(by = ["Pontos (total)", "Qtd vitórias (total)", "Saldo de gols", "Gols pró"], ascending = False)
tabela_brasileirao_final = tabela_brasileirao_final.reset_index().reset_index()
tabela_brasileirao_final["Posição"] = tabela_brasileirao_final["index"] + 1
tabela_brasileirao_final = tabela_brasileirao_final.set_index("Posição").drop("index", axis = 1)

tabela_brasileirao_final

<ipython-input-80-772cf55e4342>:80: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  tabela_brasileirao_final = tabela_brasileirao_temp.groupby("Time").sum() \


,Time,Pontos (total),Qtd jogos (total),Qtd vitórias (total),Qtd empates (total),Qtd derrotas (total),Gols pró,Gols contra,Saldo de gols,% aproveitamento
Posição,,,,,,,,,,
1,Botafogo,33.0,13.0,11.0,0.0,2.0,22.0,7.0,15.0,0.846154
2,Grêmio,26.0,13.0,8.0,2.0,3.0,24.0,18.0,6.0,0.666667
3,Flamengo,25.0,13.0,8.0,1.0,4.0,26.0,17.0,9.0,0.641026
4,Palmeiras,23.0,13.0,6.0,5.0,2.0,24.0,12.0,12.0,0.589744
5,RB Bragantino,23.0,13.0,6.0,5.0,2.0,22.0,14.0,8.0,0.589744
6,Fluminense,21.0,13.0,6.0,3.0,4.0,19.0,14.0,5.0,0.538462
7,São Paulo,21.0,13.0,6.0,3.0,4.0,18.0,13.0,5.0,0.538462
8,Internacional,21.0,13.0,6.0,3.0,4.0,13.0,14.0,-1.0,0.538462
9,Athletico PR,20.0,13.0,6.0,2.0,5.0,18.0,17.0,1.0,0.512821


In [81]:
# tabela_brasileirao_final.to_excel("Tabela Brasileirão.xlsx")
tabela_brasileirao_final.to_excel('/content/drive/My Drive/Estudo/Data Science/Análise Campeonato Brasileiro/Tabela Brasileirão.xlsx')

# Análise exploratória

# Preparação de dados

## One hot encoding

## Balanceamento

## Separação treino e teste

## Normalização

# Classificação/Regressão

# TO-DO